In [ ]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

# List collections
print(client.get_collections())

# Check info collection
info = client.get_collection("chat_history")
print(info)


collections=[CollectionDescription(name='rag_knowledge'), CollectionDescription(name='chat_history')]
status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=2 segments_count=6 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_thr

In [5]:
from qdrant_client import QdrantClient
from pprint import pprint

client = QdrantClient(host="localhost", port=6333)

def inspect_qdrant():
    collections = client.get_collections().collections
    print(f"Found {len(collections)} collections:")
    
    for col in collections:
        print("\n" + "="*40)
        print(f"Collection name: {col.name}")
        try:
            info = client.get_collection(col.name)
            print(f"Points count: {info.points_count}")
            vectors = info.config.params.vectors
            if isinstance(vectors, dict):
                print("Vector names & sizes:")
                for name, vcfg in vectors.items():
                    print(f"  - {name}: size={vcfg.size}, distance={vcfg.distance}")
            else:
                print(f"Vector config: size={vectors.size}, distance={vectors.distance}")
        except Exception as e:
            print("Could not fetch collection info:", e)
        
        print("Sample points (up to 10):")
        try:
            resp, _ = client.scroll(col.name, limit=10, with_payload=True)
            points = getattr(resp, "result", resp)
            for p in points:
                vector_names = list(p.vector.keys()) if isinstance(p.vector, dict) else ["default"]
                print(f"  id={p.id}, vector_names={vector_names}, payload={p.payload}")
        except Exception as e:
            print("Could not fetch points:", e)

if __name__ == "__main__":
    inspect_qdrant()


Found 1 collections:

Collection name: collection
Points count: 0
Vector config: size=384, distance=Cosine
Sample points (up to 10):


In [19]:
from qdrant_client import QdrantClient

# Kết nối Qdrant (ví dụ chạy local)
client = QdrantClient("http://localhost:6333")

collection_name = "collection"

# Lấy thông tin collection
info = client.get_collection(collection_name)
print("Tổng số vector:", info.vectors_count)


Tổng số vector: None


In [7]:
import numpy as np

# Giả sử bạn có embedding vector test 
dummy_vector = np.random.rand(384).tolist()

result = client.search(
    collection_name=collection_name,
    query_vector=dummy_vector,
    limit=3
)

print("Kết quả tìm được:", result)


Kết quả tìm được: []


C:\Users\admin\AppData\Local\Temp\ipykernel_14208\2745754574.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  result = client.search(


In [2]:
import qdrant_client

def check_collections():
    client = qdrant_client.QdrantClient(host="localhost", port=6333)

    collections = client.get_collections()
    print("📂 Danh sách collections trong Qdrant:")
    for coll in collections.collections:
        name = coll.name

        # Dùng count để chắc chắn có số vector
        count = client.count(collection_name=name, exact=True).count
        print(f" - {name}: {count} vectors")

    if not collections.collections:
        print("⚠️ Không có collection nào trong Qdrant (rỗng).")

if __name__ == "__main__":
    check_collections()


📂 Danh sách collections trong Qdrant:
 - collection: 0 vectors


In [14]:
from qdrant_client import QdrantClient

client = QdrantClient("http://localhost:6333")

# Xem các collection hiện có
print(client.get_collections())

# Nếu có collection "documents", xem thử số lượng points
print(client.count(collection_name="collection", exact=True))

# Lấy vài point mẫu
points = client.scroll(collection_name="collection", limit=5)
print(points)


collections=[CollectionDescription(name='collection')]
count=1
([Record(id=351531, payload={'source': 'app'}, vector=None, shard_key=None, order_value=None)], None)


In [4]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://cafef.vn"


def get_url(page: int) -> str:
    return f"{BASE_URL}/doc-nhanh.chn" if page == 1 else f"{BASE_URL}/doc-nhanh/trang-{page}.chn"


def get_article_content(link: str) -> str:
    """Vào trang chi tiết và lấy toàn bộ nội dung"""
    resp = requests.get(link, timeout=10)
    resp.encoding = "utf-8"
    soup = BeautifulSoup(resp.text, "html.parser")

    content_div = soup.select_one("div.detail-content")
    if not content_div:
        return ""

    # Ghép các đoạn văn lại
    paragraphs = [p.get_text(strip=True) for p in content_div.find_all("p") if p.get_text(strip=True)]
    return "\n".join(paragraphs)


def crawl_cafef(max_pages: int = 1):
    articles = []
    for page in range(1, max_pages + 1):
        url = get_url(page)
        resp = requests.get(url, timeout=10)
        resp.encoding = "utf-8"
        soup = BeautifulSoup(resp.text, "html.parser")

        blocks = soup.select("div.nv-text-cont")
        for block in blocks:
            link_tag = block.select_one("a.news-title")
            if not link_tag:
                continue

            href = link_tag["href"]
            article_id = href.split("-")[-1].replace(".chn", "")
            title = link_tag.get("title") or link_tag.text.strip()
            link = BASE_URL + href

            details = block.find_next_sibling("div", class_="nv-details")
            short_content = details.select_one("div.abs").text.strip() if details else ""
            time_tag = details.select_one("span.time")
            time_text = time_tag.get_text(strip=True) if time_tag else ""

            # Lấy toàn bộ nội dung trong link
            full_content = get_article_content(link)

            articles.append({
                "id": article_id,
                "title": title,
                "time": time_text,
                "summary": short_content,
                "link": link,
                "content": full_content
            })
    return articles


if __name__ == "__main__":
    data = crawl_cafef(max_pages=2)
    for item in data[:3]:
        print(item)


{'id': '188250924111917324', 'title': 'TP HCM: Tin vui cho người dân ở chung cư cũ', 'time': '(24/09/2025 12:41)', 'summary': 'Chung cư xây dựng trước năm 1994  sẽ được hỗ trợ cải tạo, xây dựng lại; TP HCM cũng kiểm định lại chất lượng hàng trăm chung cư cũ.', 'link': 'https://cafef.vn/tp-hcm-tin-vui-cho-nguoi-dan-o-chung-cu-cu-188250924111917324.chn', 'content': 'Chủ tịch UBND TP HCM vừa chỉ đạo triển khai Nghị quyết số 17/2025 của HĐND TP HCM quy định cơ chế ưu đãi, hỗ trợ đối với chủ đầu tư khi tham gia cải tạo, xây dựng lại nhà chung cư trên địa bàn thành phố.\nTP HCM triển khai nhiều nội dung hỗ trợ chung cư cũ xây dựng trước năm 1994.\nTheo đó, Chủ tịch UBND TP HCM giao Sở Tài chính chủ trì, hướng dẫn Sở Xây dựng là cơ quan được bố trí dự toán kinh phí từ nguồn vốn chi thường xuyên ngân sách cấp thành phố để thực hiện cơ chế theo Nghị quyết 17/2025.\nUBND 42 phường có chung cư xây dựng trước năm 1994 trên địa bàn thông tin đến tổ chức, cá nhân tại vị trí, khu vực có chung cư thuộ

In [ ]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://cafef.vn"


def get_url(page: int) -> str:
    """Sinh URL cho từng trang tin nhanh."""
    return f"{BASE_URL}/doc-nhanh.chn" if page == 1 else f"{BASE_URL}/doc-nhanh/trang-{page}.chn"


def get_article_content(link: str) -> str:
    """Vào trang chi tiết và lấy toàn bộ nội dung bài viết."""
    try:
        resp = requests.get(link, timeout=10)
        resp.encoding = "utf-8"
    except Exception as e:
        print(f"[Crawler] Lỗi khi request {link}: {e}")
        return ""

    if resp.status_code != 200:
        print(f"[Crawler] Lỗi {resp.status_code} khi truy cập {link}")
        return ""

    soup = BeautifulSoup(resp.text, "html.parser")

    content_div = soup.select_one("div.detail-content")
    if not content_div:
        return ""

    # Ghép các đoạn văn lại
    paragraphs = [
        p.get_text(strip=True)
        for p in content_div.find_all("p")
        if p.get_text(strip=True)
    ]
    return "\n".join(paragraphs)


def crawl_cafef(max_pages: int = 1):
    """Crawl tin nhanh trên CafeF, trả về list dict bài viết."""
    articles = []
    for page in range(1, max_pages + 1):
        url = get_url(page)
        try:
            resp = requests.get(url, timeout=10)
            resp.encoding = "utf-8"
        except Exception as e:
            print(f"[Crawler] Lỗi khi request {url}: {e}")
            continue

        if resp.status_code != 200:
            print(f"[Crawler] Lỗi {resp.status_code} khi truy cập {url}")
            continue

        soup = BeautifulSoup(resp.text, "html.parser")

        # Mỗi bài trong block fast-news
        items = soup.select("div.foreverblock.list-fast-news div.item")
        for item in items:
            nv_text = item.select_one("div.nv-text-cont")
            if not nv_text:
                continue

            link_tag = nv_text.select_one("a.news-title")
            if not link_tag:
                continue

            href = link_tag["href"]
            article_id = item.get("data-id") or href.split("-")[-1].replace(".chn", "")
            title = link_tag.get("title") or link_tag.text.strip()
            link = BASE_URL + href if href.startswith("/") else href

            details = item.select_one("div.nv-details")
            summary = (
                details.select_one("div.abs").get_text(strip=True)
                if details and details.select_one("div.abs")
                else ""
            )
            time_text = (
                details.select_one("span.time").get_text(strip=True)
                if details and details.select_one("span.time")
                else ""
            )

            # Crawl nội dung chi tiết
            full_content = get_article_content(link)

            articles.append(
                {
                    "id": article_id,
                    "title": title,
                    "time": time_text,
                    "summary": summary,
                    "link": link,
                    "content": full_content,
                }
            )

    print(f"[Crawler] Crawled {len(articles)} articles")
    return articles


if __name__ == "__main__":
    data = crawl_cafef(max_pages=2)
    for item in data[:2]:
        print(item)


In [7]:
from qdrant_client import QdrantClient
client = QdrantClient()
print(client.get_collections())


collections=[CollectionDescription(name='collection')]


In [4]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import re

BASE_URL = "https://cafef.vn"
TIMEZONE_OFFSET = 7  # UTC+7 cho giờ Việt Nam


def normalize_time(time_tag) -> str:
    """Chuẩn hóa thời gian về format dd-mm-YYYY HH:MM:SS (UTC+7)."""
    if not time_tag:
        return ""

    # Nếu có attribute title sẵn ISO (ví dụ: 2025-09-25T08:27:00)
    if time_tag.has_attr("title"):
        try:
            dt = datetime.fromisoformat(time_tag["title"])
            return dt.strftime("%d-%m-%Y %H:%M:%S")
        except Exception:
            return time_tag["title"]

    # Nếu chỉ có text: "1 giờ trước", "5 phút trước", "2 ngày trước"
    text = time_tag.get_text(strip=True).lower()
    now = datetime.utcnow() + timedelta(hours=TIMEZONE_OFFSET)

    num = int(re.search(r"\d+", text).group()) if re.search(r"\d+", text) else 0

    if "phút" in text:
        dt = now - timedelta(minutes=num)
    elif "giờ" in text:
        dt = now - timedelta(hours=num)
    elif "ngày" in text:
        dt = now - timedelta(days=num)
    else:
        dt = now

    return dt.strftime("%d-%m-%Y %H:%M:%S")


def get_article_content(link: str) -> str:
    """Lấy nội dung chi tiết của 1 bài viết."""
    try:
        resp = requests.get(link, timeout=10)
        resp.encoding = "utf-8"
    except Exception as e:
        print(f"[Crawler] Lỗi khi request {link}: {e}")
        return ""

    if resp.status_code != 200:
        print(f"[Crawler] Lỗi {resp.status_code} khi truy cập {link}")
        return ""

    soup = BeautifulSoup(resp.text, "html.parser")
    content_div = soup.select_one("div.detail-content")
    if not content_div:
        return ""

    paragraphs = [
        p.get_text(strip=True)
        for p in content_div.find_all("p")
        if p.get_text(strip=True)
    ]
    return "\n".join(paragraphs)


def crawl_cafef_stock(max_pages: int = 1):
    """Crawl tin tức Thị trường chứng khoán trên CafeF."""
    articles = []
    for page in range(1, max_pages + 1):
        url = (
            f"{BASE_URL}/thi-truong-chung-khoan.chn"
            if page == 1
            else f"{BASE_URL}/thi-truong-chung-khoan/trang-{page}.chn"
        )

        try:
            resp = requests.get(url, timeout=10)
            resp.encoding = "utf-8"
        except Exception as e:
            print(f"[Crawler] Lỗi khi request {url}: {e}")
            continue

        if resp.status_code != 200:
            print(f"[Crawler] Lỗi {resp.status_code} khi truy cập {url}")
            continue

        soup = BeautifulSoup(resp.text, "html.parser")
        items = soup.select("div.tlitem.box-category-item")

        for item in items:
            link_tag = item.select_one("h3 > a")
            if not link_tag:
                continue

            href = link_tag.get("href", "")
            title = link_tag.get("title") or link_tag.text.strip()
            link = BASE_URL + href if href.startswith("/") else href
            article_id = item.get("data-id") or href.split("-")[-1].replace(".chn", "")

            summary_tag = item.select_one("p.sapo") or item.select_one("p.box-category-sapo")
            summary = summary_tag.get_text(strip=True) if summary_tag else ""

            time_tag = item.select_one("span.time")
            time_text = normalize_time(time_tag)

            # Nội dung chi tiết
            full_content = get_article_content(link)

            articles.append({
                "id": article_id,
                "title": title,
                "time": time_text,
                "summary": summary,
                "url": link,
                "content": full_content,
                "source": "cafef"
            })

    print(f"[Crawler] Crawled {len(articles)} articles")
    return articles


if __name__ == "__main__":
    data = crawl_cafef_stock(max_pages=1)
    for item in data[:3]:
        print(item)


[Crawler] Crawled 22 articles
{'id': '188250925011238282', 'title': 'Nhà sáng lập FinPeace chỉ ra bí kíp đầu tư chứng khoán “không mất ngủ”', 'time': '25-09-2025 07:00:00', 'summary': 'Theo ông Nguyễn Tuấn Anh – nhà sáng lập FinPeace, thành công của đầu tư không chỉ đo thường bằng tỷ suất sinh lời, mà có thể được tính bằng những giấc ngủ ngon, không bị ác mộng bởi tiền bạc.', 'url': 'https://cafef.vn/nha-sang-lap-finpeace-chi-ra-bi-kip-dau-tu-chung-khoan-khong-mat-ngu-188250925011238282.chn', 'content': 'Trong những giai đoạn thị trường chứng khoán biến động mạnh, không ít nhà đầu tư rơi vào trạng thái thấp thỏm, lo âu, thậm chí mất ngủ vì danh mục “đỏ lửa”. Những cơn “ác mộng” thua lỗ trở thành nỗi ám ảnh thường trực, khiến đầu tư đôi khi không còn là hành trình tìm kiếm sự an tâm tài chính mà lại trở thành gánh nặng tinh thần.\n“Đầu tư mà tối về còn gặp ác mộng thì kể cả người giàu cũngkhóc mất!”,đó là những chia sẻ của ông Nguyễn Tuấn Anh, Nhà sáng lập FinPeace tại talkshow "The Inv

In [17]:
from qdrant_client import QdrantClient

def list_vectors_in_qdrant(collection_name: str = "my_collection", host: str = "localhost", port: int = 6333, limit: int = 100):
    """
    Liệt kê tất cả vectors trong collection và in ID + payload + độ dài vector.
    """
    client = QdrantClient(host=host, port=port)

    try:
        points, next_page = client.scroll(
            collection_name=collection_name,
            limit=limit
        )

        if not points:
            print(f"Collection '{collection_name}' hiện chưa có vector nào.")
        else:
            print(f"Collection '{collection_name}' có {len(points)} vector (lấy tối đa {limit}):")
            for point in points:
                print(f"ID: {point.id}, payload: {point.payload}, vector length: {len(point.vector) if point.vector else 0}")

            # nếu còn next_page thì nghĩa là collection còn nhiều vector nữa
            if next_page:
                print("⚠️ Collection còn vector khác, hãy tăng limit hoặc tiếp tục scroll với offset.")
    except Exception as e:
        print("Lỗi khi truy vấn Qdrant:", e)

# Test
list_vectors_in_qdrant(collection_name="collection")


Collection 'collection' có 12 vector (lấy tối đa 100):
ID: 86651, payload: {'source': 'app'}, vector length: 0
ID: 175468, payload: {'source': 'app'}, vector length: 0
ID: 190315, payload: {'source': 'app'}, vector length: 0
ID: 236817, payload: {'source': 'app'}, vector length: 0
ID: 318901, payload: {'source': 'app'}, vector length: 0
ID: 351531, payload: {'source': 'app'}, vector length: 0
ID: 462858, payload: {'source': 'app'}, vector length: 0
ID: 534956, payload: {'source': 'app'}, vector length: 0
ID: 586207, payload: {'source': 'app'}, vector length: 0
ID: 603741, payload: {'source': 'app'}, vector length: 0
ID: 798773, payload: {'source': 'app'}, vector length: 0
ID: 845293, payload: {'source': 'app'}, vector length: 0


In [ ]:
from modules.utils.qdrant_utils import add_doc, update_doc, delete_doc, get_doc, search_hybrid

# Thêm tài liệu
pid = add_doc(
    collection_name="test_hybrid",
    payload={"title": "Python", "content": "Python là ngôn ngữ lập trình."}
)

# Cập nhật payload
update_doc("test_hybrid", pid, {"category": "programming"})

# Lấy payload
print(get_doc("test_hybrid", pid))

# Hybrid search
print(search_hybrid("test_hybrid", [0.1] * 384, "ngôn ngữ lập trình Python"))

# Xóa document
delete_doc("test_hybrid", pid)


In [3]:
from modules.utils.services import qdrant_services
print(qdrant_services.client.get_collection("test_hybrid"))


status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=1 segments_count=6 config=CollectionConfig(params=CollectionParams(vectors={'binary': VectorParams(size=384, distance=<Distance.DOT: 'Dot'>, hnsw_config=None, quantization_config=None, on_disk=True, datatype=None, multivector_config=None), 'dense_vector': VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None)}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors={'sparse_vector': SparseVectorParams(index=None, modifier=None)}), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, 

In [2]:
from modules.utils.qdrant_utils import add_doc, get_doc, search_hybrid, delete_doc
from modules.utils.services import embedder_services
corpus = [
    "Python là ngôn ngữ lập trình.",
    "Ngôn ngữ C++ hiệu suất cao.",
    "Java phổ biến cho ứng dụng doanh nghiệp."
]
embedder_services.fit_bm25(corpus)
pid = add_doc("test_hybrid", [0.1]*384, None, {"title": "Python", "content": "Python là ngôn ngữ lập trình."})
print(get_doc("test_hybrid", pid))
print(search_hybrid("test_hybrid", "ngôn ngữ lập trình Python"))
# delete_doc("test_hybrid", pid)


{'title': 'Python', 'content': 'Python là ngôn ngữ lập trình.'}
[{'id': '721c345a-5277-4da4-8b8c-c4eefdb02824', 'score': 1.0, 'payload': {'title': 'Python', 'content': 'Python là ngôn ngữ lập trình.'}}]


In [4]:
import rank_bm25
print(rank_bm25.__file__)


c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\rank_bm25.py


In [3]:
import sys
print(sys.executable)  # đường dẫn python đang chạy
!{sys.executable} -m pip install rank-bm25


c:\Users\admin\AppData\Local\Programs\Python\Python311\python.exe
  Using cached rank_bm25-0.2.2-py3-none-any.whl.metadata (3.2 kB)
Using cached rank_bm25-0.2.2-py3-none-any.whl (8.6 kB)


In [ ]:
from qdrant_client import QdrantClient

def main():
    client = QdrantClient(host="localhost", port=6333)

    try:
        # Lấy danh sách collection
        collections = client.get_collections()
        print("📂 Collections hiện có trong Qdrant:")
        for c in collections.collections:
            print(f" - {c.name}")

        # Kiểm tra collection cafef_articles
        coll_name = "cafef_articles"
        print(f"\n🔍 Kiểm tra collection `{coll_name}`...")
        info = client.get_collection(coll_name)
        print(f" Trạng thái: {info.status}")
        print(f" Số vector đã index: {info.indexed_vectors_count}")
        print(f" Tổng số points: {info.points_count}")

        # Đếm chính xác số points
        count = client.count(collection_name=coll_name, exact=True)
        print(f" ✅ Count API: {count.count} points")

        if count.count == 0:
            print("⚠️  Collection rỗng (chưa có dữ liệu). Kiểm tra ingestion container!")
        else:
            print("🎉 Dữ liệu đã được upload thành công vào Qdrant!")

    except Exception as e:
        print(f"❌ Lỗi khi kết nối tới Qdrant: {e}")


if __name__ == "__main__":
    main()


In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)
client.delete_collection("cafef_articles")  # Xóa collection "cafef_articles"


True

In [25]:
from transformers.models.auto.configuration_auto import CONFIG_MAPPING
from transformers.configuration_utils import PretrainedConfig

# Hack: Đăng ký qwen3 như causal LM
CONFIG_MAPPING.register("qwen3", PretrainedConfig)


In [3]:
from modules.core.state import GlobalState
from modules.nodes.embedder import embed_query
from modules.nodes.vector_db import search_vector_db
from modules.nodes.retriever import retrieve_documents
from modules.nodes.prompt_builder import build_prompt


def run_test_pipeline(user_query: str):
    print("=== START TEST PIPELINE ===\n")

    # 1. Khởi tạo state
    state = GlobalState()
    state.user_query = user_query
    print(f"[User Query] {state.user_query}\n")

    # 2. Embedding query
    state = embed_query(state)
    print("=== EMBEDDING DONE ===")
    if hasattr(state, "query_embedding"):
        print(f"Embedding keys: {list(state.query_embedding.keys())}\n")

    # 3. Search trong Qdrant
    state = search_vector_db(state, top_k=5)
    print("=== SEARCH VECTOR DB DONE ===")
    print(f"Search results count: {len(state.search_results)}\n")

    # 4. Retrieve documents từ search_results
    state = retrieve_documents(state, max_chars=1500)
    print("=== RETRIEVER DONE ===")
    print(f"Retrieved docs: {len(state.retrieved_docs)}")
    for d in state.retrieved_docs[:3]:  # In preview 3 doc đầu
        print(f"- {d.get('title')} | {d.get('time')} | score={d.get('score')}")
    print()

    # 5. Build prompt từ retriever
    state = build_prompt(state)
    print("=== PROMPT BUILT ===")
    print(f"Prompt length: {len(state.prompt)} chars\n")

    print("=== FINAL PROMPT PREVIEW ===")
    print(state.prompt[:800])  # In thử 800 ký tự đầu

    return state


if __name__ == "__main__":
    run_test_pipeline("Hôm nay có tin tức gì mới?")


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Collection `cafef_articles` đã tồn tại.


Device set to use cuda:0


EmbedderServices device: cuda
=== START TEST PIPELINE ===

[User Query] Hôm nay có tin tức gì mới?

=== EMBEDDING DONE ===
Embedding keys: ['dense_vector', 'sparse_vector']



2025-10-05 18:33:45,906 [DEBUG] httpcore.connection: connect_tcp.started host='localhost' port=6333 local_address=None timeout=5.0 socket_options=None
2025-10-05 18:33:45,909 [DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000023185112050>
2025-10-05 18:33:45,909 [DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'POST']>
2025-10-05 18:33:45,912 [DEBUG] httpcore.http11: send_request_headers.complete
2025-10-05 18:33:45,913 [DEBUG] httpcore.http11: send_request_body.started request=<Request [b'POST']>
2025-10-05 18:33:45,914 [DEBUG] httpcore.http11: send_request_body.complete
2025-10-05 18:33:45,915 [DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'POST']>
2025-10-05 18:33:45,920 [DEBUG] httpcore.http11: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'transfer-encoding', b'chunked'), (b'content-encoding', b'br'), (b'content-type', b'application/j

=== SEARCH VECTOR DB DONE ===
Search results count: 5

=== RETRIEVER DONE ===
Retrieved docs: 5
- Con trai Chủ tịch City Auto không mua 2,4 triệu cổ phiếu CTF đã đăng ký | 05-10-2025 10:27:00 | score=0.7524984
- Con trai Chủ tịch City Auto không mua 2,4 triệu cổ phiếu CTF đã đăng ký | 05-10-2025 10:27:00 | score=0.7524984
- Con trai Chủ tịch City Auto không mua 2,4 triệu cổ phiếu CTF đã đăng ký | 05-10-2025 10:27:00 | score=0.75249835

- Ưu tiên trả lời bằng ngôn ngữ 'unknown'.

## Instruction:
 ***Bạn là trợ lý AI chuyên về tài chính. Hãy thực hiện các nhiệm vụ sau:***
1. Tóm tắt tin tức chứng khoán (cổ phiếu, chỉ số, ngành nghề) Việt Nam.
2. Phân tích xu hướng thị trường dựa trên dữ liệu và tin tức Việt Nam.
3. Cung cấp thông tin mã cổ phiếu ở Việt Nam.
4. Đưa ra gợi ý đầu tư theo hướng tham khảo, KHÔNG phải lời khuyên tuyệt đối.



## Constraints:
 ***Các quy tắc ứng xử:***
1. Không sử dung ngôn ngữ lập trình, mã code(python, js, html, markdown...), các câu lệnh (if/else,print,...).

In [4]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce GTX 1650


In [21]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="C:/Users/admin/Desktop/TT/rag_note/.env")
print("KEY:", os.getenv("OPENWEATHER_API_KEY"))


KEY: 12f0ad3507c21c7cb4a41bda00daa964


In [4]:
def get_weather(city: str = "Hanoi", country: str = "VN"):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city},{country}&appid={OPENWEATHER_API_KEY}&units=metric&lang=vi"
    print("DEBUG URL:", url)  # In URL kiểm tra

    r = requests.get(url)
    print("DEBUG Status:", r.status_code)
    print("DEBUG Response:", r.text)

    if r.status_code == 200:
        data = r.json()
        return {
            "location": f"{city}, {country}",
            "temp": data["main"]["temp"],
            "desc": data["weather"][0]["description"]
        }
    return {"error": "Không lấy được dữ liệu thời tiết."}


In [4]:
from modules.core.state import GlobalState
from modules.nodes.processor import processor_instance
from modules.api.time_api import get_now

def test_time_filter():
    state = GlobalState()
    state.user_query = "hôm nay có tin tức gì mới?"

    # chạy detect_time_filter trực tiếp
    time_range = processor_instance.detect_time_filter(state.user_query)
    print("=== TEST TIME FILTER ===")
    if time_range:
        start, end = time_range
        print("Start:", start, "| tzinfo:", start.tzinfo)
        print("End:  ", end,   "| tzinfo:", end.tzinfo)
    else:
        print("Không detect được time_filter")

if __name__ == "__main__":
    print("Current now():", get_now())
    test_time_filter()


Current now(): 2025-10-03 09:49:46.123332+07:00
=== TEST TIME FILTER ===
Start: 2025-10-03 00:00:00+07:00 | tzinfo: Asia/Ho_Chi_Minh
End:   2025-10-03 23:59:59+07:00 | tzinfo: Asia/Ho_Chi_Minh


In [2]:
from modules.core.state import GlobalState
from modules.nodes.embedder import embed_query
from modules.nodes.vector_db import search_vector_db
from modules.nodes.retriever import retrieve_documents
from modules.nodes.processor import processor_query
from datetime import datetime
import pytz

def test_retriever(user_query: str):
    print("=== TEST RETRIEVER ===")
    vn_tz = pytz.timezone("Asia/Ho_Chi_Minh")
    now = datetime.now(vn_tz)
    print(f"[Now] {now}")

    # 1. Khởi tạo state
    state = GlobalState()
    state.user_query = user_query

    # 2. Processor detect time_filter
    state = processor_query(state)
    if state.time_filter:
        start, end = state.time_filter
        print(f"[Time Filter] Start: {start} | End: {end}")

    # 3. Embedding
    state = embed_query(state)

    # 4. Search trong Qdrant
    state = search_vector_db(state, top_k=5)
    print(f"[Search] Found {len(state.search_results)} raw results")

    # 5. Retrieve docs
    state = retrieve_documents(state, max_chars=1500)

    # 6. In kết quả
    print(f"[Retriever] Retrieved {len(state.retrieved_docs)} docs")
    for d in state.retrieved_docs:
        print(f"- {d['title']} | {d['time']} | score={d['score']:.3f}")

    return state

if __name__ == "__main__":
    test_retriever("Hôm nay có tin tức gì mới?")


2025-10-05 19:03:34,837 [DEBUG] httpcore.connection: connect_tcp.started host='localhost' port=6333 local_address=None timeout=5.0 socket_options=None
2025-10-05 19:03:34,839 [DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000020A44843250>
2025-10-05 19:03:34,839 [DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'POST']>
2025-10-05 19:03:34,840 [DEBUG] httpcore.http11: send_request_headers.complete
2025-10-05 19:03:34,840 [DEBUG] httpcore.http11: send_request_body.started request=<Request [b'POST']>
2025-10-05 19:03:34,844 [DEBUG] httpcore.http11: send_request_body.complete
2025-10-05 19:03:34,845 [DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'POST']>
2025-10-05 19:03:34,846 [DEBUG] httpcore.http11: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'transfer-encoding', b'chunked'), (b'vary', b'accept-encoding, Origin, Access-Control-Request-Met

=== TEST RETRIEVER ===
[Now] 2025-10-05 19:03:34.820565+07:00
[Time Filter] Start: 1759597200 | End: 1759683599
[Search] Found 5 raw results
[Retriever] Retrieved 5 docs
- Con trai Chủ tịch City Auto không mua 2,4 triệu cổ phiếu CTF đã đăng ký | 05-10-2025 10:27:00 | score=0.749
- Con trai Chủ tịch City Auto không mua 2,4 triệu cổ phiếu CTF đã đăng ký | 05-10-2025 10:27:00 | score=0.749
- Hoa Sen Group giải thể một công ty liên kết do hoạt động không hiệu quả | 05-10-2025 16:30:00 | score=0.744
- Hoa Sen Group giải thể một công ty liên kết do hoạt động không hiệu quả | 05-10-2025 16:30:00 | score=0.744
- Con trai Chủ tịch City Auto không mua 2,4 triệu cổ phiếu CTF đã đăng ký | 05-10-2025 10:27:00 | score=0.744


In [6]:
from modules.core.state import GlobalState
from modules.nodes.prompt_builder import build_prompt
from modules.utils.services import qdrant_services
from qdrant_client import models
from datetime import datetime, timedelta
import pytz

def test_prompt_from_qdrant(top_k: int = 5):
    """
    Kiểm tra build_prompt() bằng dữ liệu thật từ Qdrant.
    """

    print("🔍 Đang truy vấn dữ liệu thực từ Qdrant...\n")

    # --- 1️⃣ Lọc tin trong 3 ngày gần nhất ---
    vn_tz = pytz.timezone("Asia/Ho_Chi_Minh")
    now_vn = datetime.now(vn_tz)
    start_ts = int((now_vn - timedelta(days=3)).timestamp())
    end_ts = int(now_vn.timestamp())

    search_filter = models.Filter(
        must=[
            models.FieldCondition(
                key="time_ts",
                range=models.Range(gte=start_ts, lte=end_ts)
            )
        ]
    )

    # --- 2️⃣ Lấy dữ liệu thật từ Qdrant ---
    results = qdrant_services.client.scroll(
        collection_name=qdrant_services.collection_name,
        scroll_filter=search_filter,
        limit=top_k,
        with_payload=True
    )

    points = results[0] if results else []
    if not points:
        print("❌ Không có dữ liệu trong Qdrant (3 ngày gần nhất).")
        return

    print(f"✅ Lấy được {len(points)} tài liệu.\n")

    # --- 3️⃣ Chuyển dữ liệu sang dạng retrieved_docs ---
    docs = []
    for p in points:
        payload = p.payload or {}
        docs.append({
            "id": p.id,
            "score": 1.0,  # scroll không trả score, có thể để mặc định 1.0
            "title": payload.get("title", ""),
            "time": payload.get("time", ""),
            "url": payload.get("url", "") or payload.get("link", ""),
            "content": payload.get("content", "") or payload.get("summary", "")
        })

    # --- 4️⃣ Tạo GlobalState và build prompt ---
    state = GlobalState(
        user_query="Hôm nay có tin tức gì mới?",
        lang="vi",
        retrieved_docs=docs
    )

    state = build_prompt(state)

    # --- 5️⃣ In kết quả ---
    print("=" * 120)
    print("🧩 PROMPT ĐƯỢC SINH RA:")
    print("=" * 120)
    print(state.prompt)
    print("=" * 120)

    # --- 6️⃣ Kiểm tra nhanh ---
    if "score=" in state.prompt:
        print("✅ Đã hiển thị score trong Retrieved Context.")
    else:
        print("⚠️ Không thấy score — kiểm tra retriever hoặc build_prompt.")
    
    print(f"📝 Tổng số tài liệu trong prompt: {len(docs)}")
    print(f"📏 Độ dài prompt: {len(state.prompt)} ký tự")

if __name__ == "__main__":
    test_prompt_from_qdrant()


2025-10-05 19:06:24,498 [DEBUG] httpcore.connection: connect_tcp.started host='localhost' port=6333 local_address=None timeout=5.0 socket_options=None
2025-10-05 19:06:24,500 [DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000020AE6AD0650>
2025-10-05 19:06:24,500 [DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'POST']>
2025-10-05 19:06:24,502 [DEBUG] httpcore.http11: send_request_headers.complete
2025-10-05 19:06:24,504 [DEBUG] httpcore.http11: send_request_body.started request=<Request [b'POST']>
2025-10-05 19:06:24,506 [DEBUG] httpcore.http11: send_request_body.complete
2025-10-05 19:06:24,506 [DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'POST']>
2025-10-05 19:06:24,507 [DEBUG] httpcore.http11: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'transfer-encoding', b'chunked'), (b'content-encoding', b'br'), (b'vary', b'accept-encoding, Orig

🔍 Đang truy vấn dữ liệu thực từ Qdrant...

✅ Lấy được 5 tài liệu.

Không được sử dụng ngôn ngữ khác ngoài tiếng Việt.

## Instruction:
 ***Bạn là trợ lý AI chuyên về tài chính. Hãy thực hiện các nhiệm vụ sau:***
1. Tóm tắt tin tức chứng khoán (cổ phiếu, chỉ số, ngành nghề) Việt Nam.
2. Phân tích xu hướng thị trường dựa trên dữ liệu và tin tức Việt Nam.
3. Cung cấp thông tin mã cổ phiếu ở Việt Nam.
4. Đưa ra gợi ý đầu tư theo hướng tham khảo, KHÔNG phải lời khuyên tuyệt đối.



## Constraints:
 ***Các quy tắc ứng xử:***
1. Không sử dung ngôn ngữ lập trình, mã code(python, js, html, markdown...), các câu lệnh (if/else,print,...).
2. Không được thêm tiền tố như `text`, `json`, `yaml`, `tool_code` hoặc bất kỳ định dạng nào khác.
3. Trả lời trực tiếp, đúng vào câu hỏi.
4. KHÔNG thêm disclaimer kiểu "Tôi không phải chuyên gia tài chính...", heading hoặc nội dung ngoài yêu cầu.
5. Dùng thông tin từ Retrieved context nếu có, nhưng không lặp lại nguyên văn.



## Current Date/Time Context:
Hôm 

In [9]:
from modules.core.state import GlobalState
from modules.nodes.prompt_builder import build_prompt
from modules.nodes.response_generator import response_node
from modules.utils.services import qdrant_services, llm_services
from qdrant_client import models
from datetime import datetime, timedelta
import pytz
import json


def test_response_real(top_k: int = 5):
    """
    Test thật response_node() với dữ liệu Qdrant thật + LLM thật.
    """

    print("🔍 [STEP 1] Lấy dữ liệu thật từ Qdrant...\n")

    # --- Lấy tin trong 3 ngày gần nhất ---
    vn_tz = pytz.timezone("Asia/Ho_Chi_Minh")
    now_vn = datetime.now(vn_tz)
    start_ts = int((now_vn - timedelta(days=3)).timestamp())
    end_ts = int(now_vn.timestamp())

    filter_time = models.Filter(
        must=[
            models.FieldCondition(
                key="time_ts",
                range=models.Range(gte=start_ts, lte=end_ts)
            )
        ]
    )

    results = qdrant_services.client.scroll(
        collection_name=qdrant_services.collection_name,
        scroll_filter=filter_time,
        limit=top_k,
        with_payload=True
    )

    points = results[0] if results else []
    if not points:
        print("❌ Không có dữ liệu trong Qdrant (3 ngày gần nhất).")
        return

    # --- Chuyển dữ liệu sang retrieved_docs ---
    docs = []
    for p in points:
        payload = p.payload or {}
        docs.append({
            "id": p.id,
            "score": 1.0,
            "title": payload.get("title", ""),
            "time": payload.get("time", ""),
            "url": payload.get("url", "") or payload.get("link", ""),
            "content": payload.get("content", "") or payload.get("summary", "")
        })

    print(f"✅ Lấy {len(docs)} tài liệu từ Qdrant.\n")

    # --- [STEP 2] Tạo GlobalState ---
    query = "Hôm nay có tin tức gì mới?"
    state = GlobalState(
        user_query=query,
        lang="vi",
        retrieved_docs=docs
    )

    # --- [STEP 3] Build prompt ---
    print("🧠 [STEP 2] Build prompt từ retrieved_docs...\n")
    state = build_prompt(state)
    print("📋 Prompt preview (rút gọn):")
    print(state.prompt[:700], "...\n")

    # --- [STEP 4] Gọi LLM thật ---
    print("🤖 [STEP 3] Gọi LLM thật qua llm_services.generator...\n")

    # Kiểm tra llm_services đã được khởi tạo chưa
    if not getattr(llm_services, "generator", None):
        print("❌ llm_services.generator chưa được khởi tạo! Kiểm tra modules/utils/services.py")
        return

    state = response_node(state)

    # --- [STEP 5] In kết quả ---
    print("\n=== 🧩 KẾT QUẢ PHẢN HỒI ===")
    print(state.final_answer)
    print("=" * 120)
    print("📊 Thông tin debug:")
    print(json.dumps(state.debug_info, indent=2, ensure_ascii=False))
    print("=" * 120)

    # In thêm danh sách nguồn
    if getattr(state, "retrieved_docs", None):
        print("📚 Nguồn tham khảo:")
        for d in state.retrieved_docs:
            print(f"- {d['title']} ({d['time']}) | {d['url']} | score={d['score']}")


if __name__ == "__main__":
    test_response_real()


2025-10-05 19:11:55,184 [DEBUG] httpcore.connection: connect_tcp.started host='localhost' port=6333 local_address=None timeout=5.0 socket_options=None
2025-10-05 19:11:55,187 [DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000020A753C9650>
2025-10-05 19:11:55,187 [DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'POST']>
2025-10-05 19:11:55,191 [DEBUG] httpcore.http11: send_request_headers.complete
2025-10-05 19:11:55,192 [DEBUG] httpcore.http11: send_request_body.started request=<Request [b'POST']>
2025-10-05 19:11:55,193 [DEBUG] httpcore.http11: send_request_body.complete
2025-10-05 19:11:55,195 [DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'POST']>
2025-10-05 19:11:55,204 [DEBUG] httpcore.http11: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'transfer-encoding', b'chunked'), (b'content-type', b'application/json'), (b'vary', b'accept-enco

🔍 [STEP 1] Lấy dữ liệu thật từ Qdrant...

✅ Lấy 5 tài liệu từ Qdrant.

🧠 [STEP 2] Build prompt từ retrieved_docs...

Không được sử dụng ngôn ngữ khác ngoài tiếng Việt.

## Instruction:
 ***Bạn là trợ lý AI chuyên về tài chính. Hãy thực hiện các nhiệm vụ sau:***
1. Tóm tắt tin tức chứng khoán (cổ phiếu, chỉ số, ngành nghề) Việt Nam.
2. Phân tích xu hướng thị trường dựa trên dữ liệu và tin tức Việt Nam.
3. Cung cấp thông tin mã cổ phiếu ở Việt Nam.
4. Đưa ra gợi ý đầu tư theo hướng tham khảo, KHÔNG phải lời khuyên tuyệt đối.



## Constraints:
 ***Các quy tắc ứng xử:***
1. Không sử dung ngôn ngữ lập trình, mã code(python, js, html, markdown...), các câu lệnh (if/else,print,...).
2. Không được thêm tiền tố như `text`, `json`, `yaml`, `tool_code` hoặc bất kỳ định dạng nào khác.
3. Trả lời trực tiếp, đúng vào câu hỏi.
4. KHÔNG thêm disclaimer kiểu "Tôi không phải chuyên gia tài chính...", heading hoặc nội dung ngoài yêu cầu.
5. Dùng thông tin từ Retrieved context nếu có, nhưng không lặp lại

2025-10-05 19:12:22,687 [INFO] RAGPipeline: llm_status: success
2025-10-05 19:12:22,690 [INFO] RAGPipeline: route: RAG
2025-10-05 19:12:22,690 [INFO] RAGPipeline: llm_preview: Thông tin về tin tức chứng khoán Việt Nam hôm nay (Thủércoles, 05/10/2025), theo tìm kiếm, có các tin tức sau:

*   City Auto bị phạt hành chính 185 triệu đồng.
*   Công ty Cổ phần City Auto (MCK: CTF
2025-10-05 19:12:22,690 [INFO] RAGPipeline: llm_response_len: 612



=== 🧩 KẾT QUẢ PHẢN HỒI ===
Thông tin về tin tức chứng khoán Việt Nam hôm nay (Thủércoles, 05/10/2025), theo tìm kiếm, có các tin tức sau:

*   City Auto bị phạt hành chính 185 triệu đồng.
*   Công ty Cổ phần City Auto (MCK: CTF) giao dịch cổ phiếu của người nội bộ và người có liên quan.
*   Hoa Sen Group giải thể một công ty liên kết do hoạt động không hiệu quả.
*   Công ty CP Đầu tư Thành Thành Công bị phạt hành chính 185 triệu đồng.

---

## Notes:
This is a simple response based on the provided information and constraints. It fulfills the prompt's request to summarize news in Vietnamese while adhering to all specified rules.
```
📊 Thông tin debug:
{
  "prompt_length": 5356,
  "prompt_preview": "Không được sử dụng ngôn ngữ khác ngoài tiếng Việt.\n\n## Instruction:\n ***Bạn là trợ lý AI chuyên về tài chính. Hãy thực hiện các nhiệm vụ sau:***\n1. Tóm tắt tin tức chứng khoán (cổ phiếu, chỉ số, ngành nghề) Việt Nam.\n2. Phân tích xu hướng thị trường dựa trên dữ liệu và tin tức Việt Nam.

In [15]:
from modules.utils.services import llm_services
from modules.nodes.prompt_builder import build_prompt
from modules.core.state import GlobalState
from modules.utils.services import qdrant_services
from qdrant_client import models
from datetime import datetime, timedelta
import pytz, json

def test_llm_response_real(top_k: int = 3):
    # 1️⃣ Lấy dữ liệu thật từ Qdrant
    vn_tz = pytz.timezone("Asia/Ho_Chi_Minh")
    now = datetime.now(vn_tz)
    start_ts = int((now - timedelta(days=3)).timestamp())
    end_ts = int(now.timestamp())
    f = models.Filter(
        must=[models.FieldCondition(key="time_ts", range=models.Range(gte=start_ts, lte=end_ts))]
    )
    docs = []
    res = qdrant_services.client.scroll(collection_name=qdrant_services.collection_name, scroll_filter=f, limit=top_k, with_payload=True)
    for p in res[0]:
        docs.append({
            "title": p.payload.get("title", ""),
            "time": p.payload.get("time", ""),
            "url": p.payload.get("url", ""),
            "content": p.payload.get("content", ""),
            "score": 1.0
        })

    # 2️⃣ Build prompt
    state = GlobalState(user_query="Hôm nay có tin tức gì mới?", retrieved_docs=docs)
    state = build_prompt(state)
    print("=== 🔍 PROMPT ===")
    print(state.prompt[:1000], "...\n")

    # 3️⃣ Gọi LLM thật
    print("=== 🤖 RAW OUTPUTS ===")
    try:
        outputs = llm_services.generator(
            state.prompt,
            max_new_tokens=300,
            do_sample=True,
            temperature=0.7,
            repetition_penalty=1.1,
            return_full_text=True,
        )
        print(json.dumps(outputs, indent=2, ensure_ascii=False))
    except Exception as e:
        print("❌ Lỗi khi gọi generator:", str(e))

if __name__ == "__main__":
    test_llm_response_real()


2025-10-05 19:31:31,410 [DEBUG] httpcore.connection: connect_tcp.started host='localhost' port=6333 local_address=None timeout=5.0 socket_options=None
2025-10-05 19:31:31,413 [DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000020AE6AAB710>
2025-10-05 19:31:31,414 [DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'POST']>
2025-10-05 19:31:31,416 [DEBUG] httpcore.http11: send_request_headers.complete
2025-10-05 19:31:31,418 [DEBUG] httpcore.http11: send_request_body.started request=<Request [b'POST']>
2025-10-05 19:31:31,419 [DEBUG] httpcore.http11: send_request_body.complete
2025-10-05 19:31:31,420 [DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'POST']>
2025-10-05 19:31:31,422 [DEBUG] httpcore.http11: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'transfer-encoding', b'chunked'), (b'content-encoding', b'br'), (b'vary', b'accept-encoding, Orig

- Ưu tiên trả lời bằng ngôn ngữ 'unknown'.

## Instruction:
 ***Bạn là trợ lý AI chuyên về tài chính. Hãy thực hiện các nhiệm vụ sau:***
1. Tóm tắt tin tức chứng khoán (cổ phiếu, chỉ số, ngành nghề) Việt Nam.
2. Phân tích xu hướng thị trường dựa trên dữ liệu và tin tức Việt Nam.
3. Cung cấp thông tin mã cổ phiếu ở Việt Nam.
4. Đưa ra gợi ý đầu tư theo hướng tham khảo, KHÔNG phải lời khuyên tuyệt đối.



## Constraints:
 ***Các quy tắc ứng xử:***
1. Không sử dung ngôn ngữ lập trình, mã code(python, js, html, markdown...), các câu lệnh (if/else,print,...).
2. Không được thêm tiền tố như `text`, `json`, `yaml`, `tool_code` hoặc bất kỳ định dạng nào khác.
3. Trả lời trực tiếp, đúng vào câu hỏi.
4. KHÔNG thêm disclaimer kiểu "Tôi không phải chuyên gia tài chính...", heading hoặc nội dung ngoài yêu cầu.
5. Dùng thông tin từ Retrieved context nếu có, nhưng không lặp lại nguyên văn.



## Current Date/Time Context:
Hôm nay là Chủ Nhật, Ngày 05 tháng 10 năm 2025, 19:31:31.
Ngày hôm qua là Ngày 

In [14]:
from modules.core.state import GlobalState
from modules.nodes.response_generator import response_node

# Giả lập mock generator
class MockLLM:
    @staticmethod
    def generator(prompt, **kwargs):
        print("🔥 PROMPT NHẬN:", prompt[:200])
        return [{"generated_text": "Thị trường hôm nay tăng nhẹ, nhóm ngân hàng dẫn đầu xu hướng."}]

from modules.utils import services
services.llm_services = MockLLM()

state = GlobalState(
    user_query="Hôm nay có tin tức gì mới?",
    prompt="Hãy tóm tắt tin tức chứng khoán Việt Nam hôm nay.",
    retrieved_docs=[
        {"title": "VN-Index tăng nhẹ", "url": "https://cafef.vn/abc", "time": "05-10-2025", "score": 0.98}
    ]
)

state = response_node(state)
print("✅ FINAL:", state.final_answer)
print("📚 SOURCES:", state.conversation_history[-1].get("sources"))


2025-10-05 19:27:27,418 [INFO] RAGPipeline: llm_status: success
2025-10-05 19:27:27,419 [INFO] RAGPipeline: route: RAG
2025-10-05 19:27:27,420 [INFO] RAGPipeline: llm_preview: **Tình hình chung:** Thị trường cổ phiếu VN3 giảm 0,5% sau khi có thông báo của Ngân hàng Nhà nước về việc siết chặt tiền tệ. Tuy nhiên, đợt này đã được đón nhận như một biện pháp cần thiết để ổn định
2025-10-05 19:27:27,421 [INFO] RAGPipeline: llm_response_len: 1176


✅ FINAL: **Tình hình chung:** Thị trường cổ phiếu VN3 giảm 0,5% sau khi có thông báo của Ngân hàng Nhà nước về việc siết chặt tiền tệ. Tuy nhiên, đợt này đã được đón nhận như một biện pháp cần thiết để ổn định hệ thống tài chính và kiểm soát lạm phát.

* **VN3 (CT2) giảm 0,48%, VND (VND) giảm 0,67%; HQC (HQC) giảm 0,91%.**
* **Bτερ (BTF) tăng 1,37%.**
* **Đầu tư vào các ngành công nghiệp năng lượng (DMP - dầu khí) vẫn đang là điểm nóng.**

**Các yếu tố quan trọng khác:**

*   **Ngân hàng nhà nước (NH):** Nhấn mạnh cho rằng siết chặt tiền tệ là cần thiết để kiềm chế lạm phát. Các quyết định này sẽ tác động đến lãi suất, tỷ giá và thanh khoản của thị trường.
*   **Cung cầu:** Sự thiếu hụt cung trong nhiều ngành nghề, đặc biệt là trong lĩnh vực bất động sản, đang góp phần làm gia tăng áp lực lên giá.
*   **Mức độ chấp nhận rủi ro:** Nhiều nhà đầu tư tỏ ra thận trọng hơn so với trước đây, do lo ngại về tình hình kinh tế vĩ mô và tiềm ẩn nguy cơ suy thoái.


**Nguồn tin chính:** Reuters, Bl

In [4]:
from modules.utils.services import qdrant_services

def check_docs_by_date(date_str: str, limit: int = 100):
    """
    Kiểm tra trong Qdrant có doc nào có payload.time chứa date_str hay không.
    Ví dụ: date_str = "03-10-2025"
    """
    scroll_res, _ = qdrant_services.client.scroll(
        collection_name=qdrant_services.collection_name,
        with_payload=True,
        limit=limit
    )

    found = []
    for r in scroll_res:
        t = (r.payload or {}).get("time", "")
        if date_str in t:
            found.append({
                "id": r.id,
                "title": r.payload.get("title"),
                "time": t,
                "url": r.payload.get("url")
            })

    if not found:
        print(f"[Check] ❌ Không tìm thấy doc nào có time chứa '{date_str}' trong {limit} docs đầu.")
    else:
        print(f"[Check] ✅ Tìm thấy {len(found)} doc(s) có time='{date_str}':")
        for d in found:
            print(d)

if __name__ == "__main__":
    check_docs_by_date("05-10-2025", limit=500)


2025-10-05 19:05:45,113 [DEBUG] httpcore.connection: connect_tcp.started host='localhost' port=6333 local_address=None timeout=5.0 socket_options=None
2025-10-05 19:05:45,116 [DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000020A44817C10>
2025-10-05 19:05:45,117 [DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'POST']>
2025-10-05 19:05:45,119 [DEBUG] httpcore.http11: send_request_headers.complete
2025-10-05 19:05:45,120 [DEBUG] httpcore.http11: send_request_body.started request=<Request [b'POST']>
2025-10-05 19:05:45,122 [DEBUG] httpcore.http11: send_request_body.complete
2025-10-05 19:05:45,124 [DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'POST']>
2025-10-05 19:05:45,127 [DEBUG] httpcore.http11: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'transfer-encoding', b'chunked'), (b'content-encoding', b'br'), (b'vary', b'accept-encoding, Orig

[Check] ✅ Tìm thấy 74 doc(s) có time='05-10-2025':
{'id': '00a75257-6bf8-4199-b75d-4f250982af6a', 'title': 'Con trai Chủ tịch City Auto không mua 2,4 triệu cổ phiếu CTF đã đăng ký', 'time': '05-10-2025 10:27:00', 'url': 'https://cafef.vn/con-trai-chu-tich-city-auto-khong-mua-24-trieu-co-phieu-ctf-da-dang-ky-188251005092826859.chn'}
{'id': '02990b21-40ed-4051-ae2d-3700aacfc44e', 'title': 'Hoa Sen Group giải thể một công ty liên kết do hoạt động không hiệu quả', 'time': '05-10-2025 16:30:00', 'url': 'https://cafef.vn/hoa-sen-group-giai-the-mot-cong-ty-lien-ket-do-hoat-dong-khong-hieu-qua-188251005155454349.chn'}
{'id': '055824a7-0900-457a-9318-2c455165c517', 'title': 'Con trai Chủ tịch City Auto không mua 2,4 triệu cổ phiếu CTF đã đăng ký', 'time': '05-10-2025 10:27:00', 'url': 'https://cafef.vn/con-trai-chu-tich-city-auto-khong-mua-24-trieu-co-phieu-ctf-da-dang-ky-188251005092826859.chn'}
{'id': '06348cbe-4477-48cf-a9a4-2fff3bf350d6', 'title': 'Góc nhìn chuyên gia: "Nín thở" chờ tin nân

In [23]:
from modules.core.state import GlobalState
from modules.nodes.processor import processor_query
from modules.nodes.embedder import embed_query
from modules.nodes.vector_db import search_vector_db
from modules.nodes.retriever import retrieve_documents
from modules.nodes.prompt_builder import build_prompt
from modules.nodes.response_generator import response_node
from datetime import datetime
import pytz


def test_full_pipeline(user_query: str, vocab=None):
    print("=== TEST FULL PIPELINE ===")
    vn_tz = pytz.timezone("Asia/Ho_Chi_Minh")
    now = datetime.now(vn_tz)
    print(f"[Now] {now.strftime('%d-%m-%Y %H:%M:%S %Z')}")

    # 1. Init state
    state = GlobalState()
    state.user_query = user_query
    print(f"[User Query] {state.user_query}")

    # 2. Processor
    state = processor_query(state, vocab=vocab)
    print(f"[Processed Query] {state.processed_query}")
    print(f"[Lang] {state.lang} | [Is Greeting] {state.is_greeting}")
    if state.time_filter:
        start, end = state.time_filter
        print(f"[Time Filter] {start} → {end}")
    if state.api_response:
        print(f"[API Response] {state.api_response}")
        return state   # dừng tại đây vì API query đã trả lời xong

    # 3. Embedder
    state = embed_query(state)
    if state.query_embedding:
        dense_len = len(state.query_embedding["dense_vector"]) if state.query_embedding.get("dense_vector") is not None else 0
        has_sparse = state.query_embedding.get("sparse_vector") is not None
        print(f"[Embed] Dense={dense_len}, Sparse={'OK' if has_sparse else 'None'}")
    else:
        print("[Embed] ❌ Không có embedding")

    # 4. Vector DB Search
    state = search_vector_db(state, top_k=5)
    print(f"[VectorDB] Found {len(state.search_results)} results")
    for r in state.search_results[:3]:
        print(f"  - {r.get('title','')} | score={r.get('score'):.3f} | time={r.get('time')}")

    # 5. Retriever
    state = retrieve_documents(state, max_chars=1500)
    print(f"[Retriever] Retrieved {len(state.retrieved_docs)} docs")
    for d in state.retrieved_docs:
        print(f"  - {d['title']} | {d['time']} | score={d['score']:.3f}")

    # 6. Build Prompt
    state = build_prompt(state)
    print("=== PROMPT PREVIEW ===")
    print(state.prompt[:600], "...\n")

    # 7. Response Node (LLM)
    state = response_node(state)
    print("=== ASSISTANT RESPONSE ===")
    print(state.final_answer)

    print("=== END PIPELINE ===")
    return state


if __name__ == "__main__":
    # ví dụ test
    test_full_pipeline("Hôm nay thị trường chứng khoán có tin gì mới?")


=== TEST FULL PIPELINE ===
[Now] 03-10-2025 12:21:20 +07
[User Query] Hôm nay thị trường chứng khoán có tin gì mới?
[Processed Query] hôm nay thị trường chứng khoán có tin gì mới
[Lang] vi | [Is Greeting] False
[Time Filter] 2025-10-03 00:00:00+07:00 → 2025-10-03 23:59:59+07:00
[Embed] Dense=384, Sparse=None
[VectorDB] Found 5 results
{'id': 'b4badc12-aae8-438d-bc46-040d48836c40', 'score': 0.7776, 'title': '‘Nhận án’ rời sàn, cổ phiếu nhóm Bamboo Capital đồng loạt lao dốc'}
{'id': 'b1540ae2-0841-4f29-980f-a5ba4c5377b5', 'score': 0.774, 'title': 'Tài sản mã hóa Việt Nam chạm mốc 220 tỷ USD'}
{'id': '147d4a55-7747-4225-a1f0-817d95c05bf0', 'score': 0.7715, 'title': 'Tiêu dùng khởi sắc: Ai sẽ tận dụng lợi thế hệ sinh thái tích hợp?'}
[VectorDB] Found 5 results
  -  | score=0.778 | time=None
  -  | score=0.774 | time=None
  -  | score=0.772 | time=None
=== RETRIEVED DOCS ===
{'id': 'b1540ae2-0841-4f29-980f-a5ba4c5377b5', 'score': 0.7740048749999999, 'title': 'Tài sản mã hóa Việt Nam chạm mố

In [1]:
### TEST WEATHER
from modules.core.state import GlobalState
from modules.nodes.processor import processor_query

def test_weather_query(user_query: str = "thời tiết Hà Nội"):
    print("=== TEST WEATHER API ===")
    # Khởi tạo state
    state = GlobalState()
    state.user_query = user_query

    # Gọi processor để check API query
    state = processor_query(state)

    print("[User Query] :", state.user_query)
    print("[Processed Query] :", getattr(state, "processed_query", None))
    print("[API Type] :", getattr(state, "api_type", None))
    print("[API Response] :", getattr(state, "api_response", None))
    print("=== END TEST ===")

    return state


if __name__ == "__main__":
    # Ví dụ test
    test_weather_query("thời tiết Hà Nội hôm nay thế nào?")
    test_weather_query("thời tiết Hồ Chí Minh")
    test_weather_query("thời tiết Huế")


=== TEST WEATHER API ===
DEBUG URL http://api.openweathermap.org/data/2.5/weather?q=Hà Nội  ?,VN&appid=12f0ad3507c21c7cb4a41bda00daa964&units=metric&lang=vi
DEBUG Status 404
DEBUG Response: {"cod":"404","message":"city not found"}
[User Query] : thời tiết Hà Nội hôm nay thế nào?
[Processed Query] : thời tiết hà nội hôm nay thế nào
[API Type] : None
[API Response] : Không lấy được dữ liệu thời tiết.
=== END TEST ===
=== TEST WEATHER API ===
DEBUG URL http://api.openweathermap.org/data/2.5/weather?q=Ho Chi Minh,VN&appid=12f0ad3507c21c7cb4a41bda00daa964&units=metric&lang=vi
DEBUG Status 200
DEBUG Response: {"coord":{"lon":106.6667,"lat":10.75},"weather":[{"id":804,"main":"Clouds","description":"mây đen u ám","icon":"04d"}],"base":"stations","main":{"temp":30.61,"feels_like":37.61,"temp_min":30.61,"temp_max":30.61,"pressure":1010,"humidity":91,"sea_level":1010,"grnd_level":1009},"visibility":10000,"wind":{"speed":1.95,"deg":145,"gust":1.84},"clouds":{"all":100},"dt":1759472892,"sys":{"coun

In [2]:
from modules.core.state import GlobalState
from modules.nodes.processor import processor_query

def test_weather_in_processor():
    queries = [
        "Thời tiết Hà Nội hôm nay thế nào?",
        "Thời tiết thành phố Hồ Chí Minh ra sao?",
        "Dự báo thời tiết TP HCM ngày mai",
        "Thời tiết Sài Gòn",
        "Thời tiết HCM",
        "Thời tiết Đà Nẵng",
        "Thời tiết Huế"
    ]

    for q in queries:
        print("\n=== TEST PROCESSOR WEATHER ===")
        state = GlobalState()
        state.user_query = q
        state = processor_query(state)

        print(f"[User Query] {q}")
        print(f"[Processed Query] {state.processed_query}")
        print(f"[API Type] {getattr(state, 'api_type', None)}")
        print(f"[API Response] {getattr(state, 'api_response', None)}")

if __name__ == "__main__":
    test_weather_in_processor()



=== TEST PROCESSOR WEATHER ===
DEBUG URL http://api.openweathermap.org/data/2.5/weather?q=Hà Nội,VN&appid=12f0ad3507c21c7cb4a41bda00daa964&units=metric&lang=vi
DEBUG Status 200
DEBUG Response: {"coord":{"lon":105.8412,"lat":21.0245},"weather":[{"id":800,"main":"Clear","description":"bầu trời quang đãng","icon":"01d"}],"base":"stations","main":{"temp":32.74,"feels_like":36.63,"temp_min":32.74,"temp_max":32.74,"pressure":1009,"humidity":53,"sea_level":1009,"grnd_level":1008},"visibility":10000,"wind":{"speed":2.92,"deg":140,"gust":3.44},"clouds":{"all":0},"dt":1759474076,"sys":{"country":"VN","sunrise":1759445286,"sunset":1759488191},"timezone":25200,"id":1581130,"name":"Hà Nội","cod":200}
[User Query] Thời tiết Hà Nội hôm nay thế nào?
[Processed Query] thời tiết hà nội hôm nay thế nào
[API Type] weather
[API Response] Thời tiết tại Hà Nội, VN: 32.74°C, bầu trời quang đãng

=== TEST PROCESSOR WEATHER ===
DEBUG URL http://api.openweathermap.org/data/2.5/weather?q=Ho Chi Minh,VN&appid=12f

In [13]:
from modules.core.state import GlobalState
from modules.nodes.router import route_intent
from modules.nodes.processor import processor_query

def test_query(q):
    state = GlobalState(user_query=q)
    state = processor_query(state)
    result = route_intent(state)
    print(f"\n🧠 Query: {q}")
    print(f"➡️ route_to = {result.route_to}")
    print(f"➡️ api_type = {result.api_type}")
    print(f"➡️ api_response = {result.api_response[:200] if result.api_response else 'None'}")

# Các case mẫu
# test_query("Xin chào")
test_query("Hôm nay có tin tức gì biến động?")
# test_query("Tin tức chứng khoán hôm nay?")
# test_query("Hôm nay có tin tức gì mới?")
# test_query("Giá cổ phiếu VCB hôm nay?")
# test_query("Phân tích cổ phiếu TCB")
# test_query("Thời tiết ở Hà Nội hôm nay?")
# test_query("Bây giờ là mấy giờ?")
# test_query("Phân tích tình hình kinh tế Việt Nam hiện nay")



🧠 Query: Hôm nay có tin tức gì biến động?
➡️ route_to = rag
➡️ api_type = new general
➡️ api_response = None
